# About the Data

The data comes from WHO and is linked to the field of demographics. Here, we demonstrate one of the ways to approach the crude births data.

For more information on Panel data:
http://dspyt.com/2021/05/27/panel-data-econometrics-an-introduction-with-an-example-in-python/

## Loading Libraries

In [ ]:
!pip install linearmodels > /dev/null
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.panel import PooledOLS, RandomEffects, BetweenOLS
import statsmodels.api as sm

## Loading Data

In [ ]:
cols2skip = [1,2,3] 
df = pd.read_csv('/kaggle/input/birth-rate/API_SP.DYN.IMRT.FE.IN_DS2_en_csv_v2_2253793.csv', skiprows=3, usecols=[i for i in range(64) if i not in cols2skip],index_col='Country Name').T

# Inspecting the Data

### Let's look at the data of the individual country: Angola

In [ ]:
df.Angola

### What about the descriptive statistic of other countries

In [ ]:
df.describe()

**Some of the columns contain full NaNs, hence it is reasonable to drop such data:**

In [ ]:
df = df.dropna(axis=1, how='all')

# Displaying the data for 10 Countries:

In [ ]:
df.iloc[:,:10].plot(subplots=True, figsize=(10,10))
plt.savefig('plot.png')
plt.show()

**From the data we can see a general downward trend in the number of crude births per 1000 people**

## Let's investigate further through Panel data analysis

In [ ]:
year = df.index
df['year'] = year.astype(int)
df = df.melt(id_vars=["year"], 
        var_name="Country", 
        value_name="Birth Rate").dropna().reset_index(drop=True)
year = df.year
df = df.set_index([df.index, 'year'])
df['year'] = pd.Categorical(year)

In [ ]:
df.head()

## Pooled OLS

In [ ]:
exog_vars = ["Country", 'year']
exog = sm.add_constant(df[exog_vars])
mod = PooledOLS(df['Birth Rate'], exog)
pooled_res = mod.fit()
print(pooled_res)

## The between estimator

In [ ]:
mod = BetweenOLS(df['Birth Rate'], exog)
re_res = mod.fit()
print(re_res)

# The downward trend is confirmed across the given countries,regions and periods.